In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [16]:
sentence = "Repeat is the best medicine for memory".split()
print(sentence)

['Repeat', 'is', 'the', 'best', 'medicine', 'for', 'memory']


In [17]:
vocab = sorted(list(set(sentence)))
print(vocab)

['Repeat', 'best', 'for', 'is', 'medicine', 'memory', 'the']


In [18]:
vocab_size = len(vocab)
vocab_size

7

In [19]:
word_to_index = {tkn:i for i, tkn in enumerate(vocab,1)}
print(word_to_index)
word_to_index['<unk>']=0
print(word_to_index)

{'Repeat': 1, 'best': 2, 'for': 3, 'is': 4, 'medicine': 5, 'memory': 6, 'the': 7}
{'Repeat': 1, 'best': 2, 'for': 3, 'is': 4, 'medicine': 5, 'memory': 6, 'the': 7, '<unk>': 0}


In [20]:
print(word_to_index['for'])         # word가 key가 되고 index가 value가 된다

3


In [21]:
encode = [word_to_index[t] for t in sentence]
print(encode)

[1, 4, 7, 2, 5, 3, 6]


In [22]:
def build_data(sentence, word_to_index):
    encode = [word_to_index[t] for t in sentence]
    input_seq = encode[:-1]
    label_seq = encode[1:]
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq = torch.LongTensor([label_seq])
    print(input_seq)
    print(label_seq)
    return input_seq, label_seq

In [23]:
X,Y = build_data(sentence, word_to_index)
print(X)
print(Y)

tensor([[1, 4, 7, 2, 5, 3]])
tensor([[4, 7, 2, 5, 3, 6]])
tensor([[1, 4, 7, 2, 5, 3]])
tensor([[4, 7, 2, 5, 3, 6]])


In [24]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()

        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size)
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=batch_first)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x):
        output = self.embedding_layer(x)
        # 임베딩층 : 크기변화(배치크기, 시퀀스 길이) => (배치크기, 시퀀스길이, 임베딩 차원)
        output, hidden = self.rnn(output)
        # RNN층 : 크기변화(배치크기, 시퀀스 길이, 임베딩 차원)
        # output : (배치크기, 시퀀스 길이, 은닉층 크기)
        # hidden : (1, 배치크기, 은닉층 크기)
        
        output = self.fc(output)
        # 크기변화 : (배치크기, 시퀀스 길이, 단어장 크기) => (배치크기*시퀀스길이, 단어장 크기)

        return output.view(-1, output.size(2))

In [25]:
vocab_size = len(word_to_index)
input_size = vocab_size
hidden_size = 2

In [26]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [27]:
index_to_word = {v:k for k,v in word_to_index.items()}
print(index_to_word)
decode = lambda y:[index_to_word.get(x) for x in y]
print(decode)

{1: 'Repeat', 2: 'best', 3: 'for', 4: 'is', 5: 'medicine', 6: 'memory', 7: 'the', 0: '<unk>'}
<function <lambda> at 0x000001CE5A6E6980>


In [45]:
for step in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_func(output, Y.view(-1))
    loss.backward()
    optimizer.step()

    pred = output.softmax(-1).argmax(-1).tolist()

    if step%10==0:
        print(step, pred, 'str: ', ' '.join(['Repeat']+decode(pred)))

0 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
10 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
20 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
30 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
40 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
50 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
60 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
70 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
80 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
90 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
100 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
110 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
120 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine best medicine best memory
130 [4, 5, 2, 5, 2, 6] str:  Repeat is medicine b